# Advent of Code 2017

In [100]:
import csv
import numpy as np
from itertools import product
import math

## Day 1

In [5]:
with open('input_day1.txt', 'r') as f:
    inp1 = f.read()
len(inp1)

2026

### 1.1

In [6]:
seq1 = [int(inp1[i:(i+1)]) for i in range(len(inp1))] + [int(inp1[-1])]
print(f"the sequence contains {len(seq1)} digits")
sum([seq1[i] for i in range(len(seq1)-1) if (seq1[i] - seq1[i+1]) == 0])

the sequence contains 2027 digits


1216

### 1.2

In [8]:
seq2 = [int(inp1[i:(i+1)]) for i in range(len(inp1))] * 2
print(f"the sequence contains {len(seq2)} digits")
step = int(len(inp1) / 2)
sum([seq2[i] for i in range(len(inp1)) if (seq2[i] - seq2[i+step]) == 0])

the sequence contains 4052 digits


1072

## Day 2

In [12]:
with open('input_day2.txt', 'r') as f:
    inp2 = [l for l in csv.reader(f)]
inp2 = np.asarray(inp2).astype(int)
inp2.shape

(16, 16)

### 2.1

In [14]:
sum(inp2.max(1) - inp2.min(1))

34925

### 2.2

In [15]:
def evenly_div(arr):
    pairs = np.asarray(list(product(arr, repeat=2)))
    div = (pairs[:, 0] / pairs[:, 1]).astype(int)
    return div[(pairs[:, 0] % pairs[:, 1] == 0) & (pairs[:, 0] != pairs[:, 1])][0]

In [16]:
sum(np.apply_along_axis(evenly_div, 1, inp2))

221

## Day 3

In [263]:
inp3 = 361527

### 3.1

In [273]:
upper_sqrt = math.ceil(math.sqrt(inp3))
edge_len = upper_sqrt + int(upper_sqrt % 2 == 0)
print(f'square root \t= {math.sqrt(inp3)} (=> {upper_sqrt} => {edge_len})')

ring_dist = int(edge_len / 2)
print(f'ring to center \t= {ring_dist}')

last_num = edge_len**2
print(f'max of ring \t= {last_num}')

square root 	= 601.2711534740379 (=> 602 => 603)
ring to center 	= 301
max of ring 	= 363609


max number of ring = 363609


In [275]:
edge_center = np.array([(last_num - int(edge_len / 2)) - (edge_len - 1) * i for i in range(4)])
edge_dist = min(abs(edge_center - inp3))
print(f'to edge center = {edge_dist}')

to edge center = 25


In [276]:
ring_dist + edge_dist

326

### 3.2